In [28]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [29]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [30]:
from ipywidgets import interact, interactive, fixed, interact_manual, Button, Layout
import ipywidgets as widgets
import matplotlib
from IPython.display import Markdown, display
import numpy.polynomial.polynomial as poly
from IPython.display import display, HTML
from matplotlib import pyplot as plt
import pandas as pd
import math
import numpy as np
import qgrid
from scipy import stats
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt



DF = pd.read_csv("https://drive.google.com/uc?id=1GtlL7gm9BnQihLuqHkbYcx_Hr1KYd2cO") # 2001-2017 multi-year weather data 4th st

DF["date"] = pd.to_datetime(DF["date"])
DF = DF.set_index(DF["date"])
DF["avgt.C"] = (DF["mint.C"] + DF["maxt.C"]) / 2
DF["radn.MJ/m2"] = DF["radn.W/m2"] * 0.0864
DF["delta"] = 4098 * (0.6106 * np.exp((17.27 * DF["avgt.C"])/(DF["avgt.C"] + 237.3))) / ((DF["avgt.C"] + 237.3)**2)
elevation = 992 # high plians of texas in meters
P = 101.3 * (((293 - (0.0065 * elevation)) / 293) ** 5.26)
gamma = 0.000665 * P
DF["delta.term"] = DF["delta"] / (DF["delta"] + (gamma * (1 + 0.34 * DF["wind.2m.m/s"])))
DF["psi.term"] = gamma / (DF["delta"] + (gamma * (1 + 0.34 * DF["wind.2m.m/s"])))
DF["temp.term"] = (900 / (DF["avgt.C"] + 237)) * DF["wind.2m.m/s"]
DF["max.sat.vap"] = 0.6108 * np.exp((17.27 * DF["maxt.C"]) / (DF["maxt.C"] + 237.3)) # maximum
DF["min.sat.vap"] = 0.6108 * np.exp((17.27 * DF["mint.C"]) / (DF["mint.C"] + 237.3)) # minimum
DF["mean.sat.vap"] = (DF["min.sat.vap"] + DF["max.sat.vap"]) / 2
DF["actual.vap"] = ((DF["min.sat.vap"] * (DF["max.RH.%"] / 100)) + (DF["max.sat.vap"] * (DF["min.RH.%"] / 100))) / 2
DF["earth.sun.rel.dis"] = 1 + (0.033 * np.cos(((2 * math.pi)/365) * DF["day"]))
DF["solar.decl"] = 0.409 * np.sin((((2 * math.pi)/365) * DF["day"]) - 1.39)
lat_in_rad = (math.pi / 180) * (33.5947684)
DF["sunset_angle"] = np.arccos(-math.tan(lat_in_rad) * np.tan(DF["solar.decl"]))
aa = DF["sunset_angle"]*np.sin(lat_in_rad)*np.sin(DF["solar.decl"])
bb = np.cos(lat_in_rad)*np.cos(DF["solar.decl"])*np.sin(DF["sunset_angle"])
DF["extraT_rad"] = (((24*60)/math.pi)*0.0820*DF["earth.sun.rel.dis"])*(aa + bb)
DF["clr_sky_rad"] = (0.75 + (2e-5)*elevation) * DF["extraT_rad"]
albedo = 0.23
DF["net.rad.MJ/m2*day"] = (1 - 0.23) * DF["radn.MJ/m2"]
sb_const = 4.903e-9
DF["sb_flux"] = ((DF["maxt.C"]+273.16)**4 + (DF["mint.C"]+273.16)**4)/2
DF["outLW_rad"] = sb_const * (DF["sb_flux"] * (0.34-(0.14*np.sqrt(DF["actual.vap"])))*(1.35*(DF["radn.MJ/m2"]/DF["clr_sky_rad"]) - 0.35))
DF["total.net.rad"] = DF["net.rad.MJ/m2*day"] - DF["outLW_rad"]
DF["total.net.rad.mm"] = DF["total.net.rad"] * 0.408
DF["ET_rad"] = DF["delta.term"] * DF["total.net.rad.mm"]
DF["ET_wind"] = DF["psi.term"]*DF["temp.term"]*(DF["mean.sat.vap"] - DF["actual.vap"])
DF["ETo"] = DF["ET_rad"] + DF["ET_wind"]
DF["num_idx"] = range(0,len(DF["date"]),1)

In [31]:
DF.iloc[:8,:]

,date,year,day,radn.W/m2,maxt.C,mint.C,soiltemp.4in.C,soiltemp.8in.C,rain.mm,wind.2m.m/s,...,clr_sky_rad,net.rad.MJ/m2*day,sb_flux,outLW_rad,total.net.rad,total.net.rad.mm,ET_rad,ET_wind,ETo,num_idx
date,,,,,,,,,,,,,,,,,,,,,
2001-01-01,2001-01-01,2001,1,55.0,-0.9,-3.7,0.7,1.5,0.0,1.80,...,13.696858,3.659040,5.383306e+09,0.758705,2.900335,1.183337,0.335745,0.112372,0.448118,0
2001-01-02,2001-01-02,2001,2,73.0,-1.6,-5.6,0.7,1.5,0.0,1.67,...,13.738400,4.856544,5.281596e+09,1.735893,3.120651,1.273225,0.346404,0.135952,0.482356,1
2001-01-03,2001-01-03,2001,3,163.0,8.2,-5.9,0.4,1.2,0.0,1.87,...,13.783234,10.844064,5.684399e+09,6.956119,3.887945,1.586281,0.520199,0.740301,1.260500,2
2001-01-04,2001-01-04,2001,4,163.0,15.2,-2.5,1.3,1.5,0.0,1.50,...,13.831345,10.844064,6.140367e+09,7.487326,3.356738,1.369549,0.578187,1.348218,1.926405,3
2001-01-05,2001-01-05,2001,5,160.0,13.0,0.3,3.2,2.6,0.0,1.18,...,13.882717,10.644480,6.148837e+09,7.085060,3.559420,1.452243,0.646263,0.885654,1.531916,4
2001-01-06,2001-01-06,2001,6,130.0,14.1,0.1,4.1,3.8,0.0,1.96,...,13.937335,8.648640,6.192517e+09,5.144805,3.503835,1.429565,0.585091,1.256566,1.841657,5
2001-01-07,2001-01-07,2001,7,157.0,8.3,-0.6,4.6,4.4,0.0,2.95,...,13.995178,10.444896,5.897307e+09,6.581033,3.863863,1.576456,0.505713,1.133833,1.639546,6
2001-01-08,2001-01-08,2001,8,163.0,11.6,-3.7,3.9,4.0,0.0,1.28,...,14.056228,10.844064,5.923664e+09,7.199445,3.644619,1.487005,0.592977,0.954031,1.547008,7


In [32]:
qgrid_widget = qgrid.QgridWidget(df=DF[["ETo","ET_wind","ET_rad","total.net.rad","total.net.rad.mm",
                                        "radn.MJ/m2","avgt.C","wind.2m.m/s"]], show_toolbar=False)
#qgrid_widget.layout = ipyw.Layout(width='40%')
qgrid_widget

QgridWidget(grid_options={'highlightSelectedCell': False, 'sortable': True, 'autoEdit': False, 'filterable': T…

In [33]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [34]:
%matplotlib inline

def plot_two_var_regress(x, y):

    d = {"x":x,"y":y}
    data_in = pd.DataFrame(d)
    slope, intercept, r_value, p_value, std_err = stats.linregress(data_in["x"], data_in["y"])

    plt.figure(figsize=(22,8))
    plt.plot(data_in["x"], data_in["y"], "go", label = "x vs y")
    plt.plot(data_in["x"], intercept + slope*data_in["x"], 'r', label='fitted line')
    plt.legend(prop={'size':12})
    plt.xlabel(str(x.name))
    plt.ylabel(str(y.name))
    plt.title(str(y.name) + " as a function of " + str(x.name))
    
    
    return plt

def get_polynomial_regression(x_in, y_in, degree_of_poly):

    x = x_in
    y = y_in

    z = np.polyfit(x, y, degree_of_poly)
    f = np.poly1d(z) # fit_fn is now a function which takes in x and returns an estimate for y
    x_new = np.linspace(x[0], x[-1], len(x))
    y_new = f(x_new)
    
    return(x_new, y_new, f, z)

first_start_date = dt.strptime('2008-01-01', '%Y-%d-%m')
first_end_date = dt.strptime('2008-12-31', "%Y-%m-%d")

x_var_names = list(DF)
y_var_names = list(DF)

setStart = widgets.DatePicker(description="Start Date: ",disabled=False,value=first_start_date)
setStop = widgets.DatePicker(description="Stop Date: ",disabled=False,value=first_end_date)
y_var = widgets.Dropdown(options=y_var_names, value='ETo',description="y: ")

items = [setStart, setStop, y_var]
widgets.HBox(items)

def plot_polynomial_regression(setStart, setStop, y_var):
    data4plot = DF.loc[setStart:setStop,(y_var, "num_idx")]
    (x_new, y_new, f, z) = get_polynomial_regression(data4plot["num_idx"], data4plot[y_var], 3)
    data4plot["y_new"] = y_new
    widgets.HBox(items)
    plt.figure(figsize=(19,10))
    plt.plot(data4plot[y_var], "go", label = y_var)
    plt.plot(data4plot["y_new"], 'r', label='fitted line')
    plt.legend(prop={'size':12})
#   plt.xlabel(str(x.name))
    plt.ylabel(y_var)
    plt.title(y_var + " as a function of time")
    plt.show    

out = widgets.interactive_output(plot_polynomial_regression, {'setStart': setStart, 'setStop': setStop, 'y_var': y_var})
ui = widgets.HBox(items)

display(ui, out)

# Select a  date 2017 or before

Output()